# M1M3 cell learning
Craig Lage - 14-Apr-23 \
The 17 tons of mirror are supported by 156 pneumatic actuators where 44 are single-axis and provide support only on the axial direction, 100 are dual-axis providing support in the axial and lateral direction, and 12 are dual-axis providing support in the axial and cross lateral directions. \
Positioning is provided by 6 hard points in a hexapod configuration which moves the mirror to a fixed operational position that shall be maintained during telescope operations. The remaining optical elements will be moved relative to this position in order to align the telescope optics. Support and optical figure correction is provided by 112 dual axis and 44 single axis pneumatic actuators. 

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LightSource as LS
import pickle as pkl
from astropy.time import Time, TimeDelta
import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable

from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('usdf_efd')
FATABLE = M1M3FATable.FATABLE

In [ ]:
# Times of bump test
start = Time("2020-11-01T00:00:00", scale='utc')
end = Time("2023-04-20T00:00:00", scale='utc')

In [ ]:
manyBumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", start, end)

In [ ]:
len(manyBumps)

In [ ]:
id = 227
theseBumps = manyBumps[manyBumps['actuatorId']==id]
index = M1M3FATable.actuatorIDToIndex(id)
primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
primaryForce = f"zForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
    if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
        secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondaryForce = f"yForce{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
        secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondaryForce = f"xForce{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
        secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
else:
    secondaryName = None
plt.subplots_adjust(wspace=0.3)
plt.suptitle(f"Multiple bump tests Actuator ID {id}", fontsize=18)

# Now find the separate tests
times = theseBumps['timestamp'].values
startTimes = []
endTimes = []
for i, time in enumerate(times):
    if i == 0:
        startTimes.append(time)
        continue
    if (time - times[i-1]) > 60.0:
        startTimes.append(time)
        endTimes.append(times[i-1])
endTimes.append(times[-1])
print(len(startTimes))
for time in startTimes:
    print(Time(time, format='unix_tai').isot)

In [ ]:
theseBumps.tail(20)

In [ ]:
manyBumps.head(30)

In [ ]:
theseBumps = manyBumps[manyBumps['actuatorId']==227]

In [ ]:
len(theseBumps)

In [ ]:
times = theseBumps['timestamp'].values
startList = []
endList = []
for i, time in enumerate(times):
    if i == 0:
        startList.append(time)
        continue
    if (time - times[i-1]) > 60.0:
        startList.append(time)
        endList.append(times[i-1])
endList.append(times[-1])        
print(len(startList), len(endList))


In [ ]:
for i in range(len(startList)):
    print(i, endList[i] - startList[i])